# Training data preparation

## Download datasets

In [ ]:
"""
%%sh
cat download_data.sh
./download_data.sh
"""

## Patch the datasets

In [2]:
from deep_learning_lab.data_preparation import Orchestrator, DataStructure

Orchestrator.DATASETS.keys()

dict_keys(['reid', 'fcr', 'bcs_a', 'bcs_b', 'bcs_e', 'bcs_h', 'bcs_n', 'bcs_s', 'bcs_u', 'bcs_un', 'bcc_a', 'bcc_b', 'bcc_bh', 'bcc_e', 'bcc_h', 'bcc_n', 'bcc_s', 'bcc_u', 'bcc_un'])

In [ ]:
sets_labels = [['ImageRegion'], ['TextLine'], ['TextRegion']] # Atomic labels are to be promoted

orc = Orchestrator(
    output_structure= DataStructure(dir_data= "training_data",
                                    dir_images= "images",
                                    dir_labels= "labels")
)

orc.ingestDatasets(
    datasets= [],
    add_defaults= True
)

for set_labels in sets_labels:
    orc.ingestLabels(
        uniform_set_labels= set_labels,
        prompt= False
    )
    orc.validate(
        auto_yes= True,
        verbose= 0
    )
    orc.preprocess(
        resize= (841, 1188), # To have 1e6 pixels and tensors of same size
        overwrite= False,
        verbose= 2
    )
    print()

# Deep learning lab

In [4]:
import deep_learning_lab.gpu_setup as gpu

gpu.cudaDeviceSelection(preselected_device= 0)
print(gpu.cudaInfo())

Torch (1.12.1)
CUDA (11.3)
GPU (RTX A6000)
CUDA memory (48.69 GB)


## Training

In [5]:
labels = ['TextLine']

In [6]:
from deep_learning_lab import model

trainer = model.Trainer(
    labels,
    workdir= "results",
    input_dir= "training_data",
    train_ratio= 0.80,
    val_ratio= 0.10
)

In [7]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

tensorboard_dir = trainer.tensorboard_dir
!echo $tensorboard_dir
#!rm -r $tensorboard_dir & mkdir -p $tensorboard_dir
%tensorboard --logdir $tensorboard_dir

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
results/TextLine/tensorboard/log


Reusing TensorBoard on port 6011 (pid 13409), started 0:01:21 ago. (Use '!kill 13409' to kill it.)

In [ ]:
trainer.train(
    batch_size= 4,
    epochs= 100,
    learning_rate= 1e-4,
    gamma_exp_lr= 0.9995,
    evaluate_every_epoch= 5,
    val_patience= 4,
    repeat_dataset= 4,
    output_size= 1e6
)

{'color_labels': {'label_json_file': 'results/TextLine/training_data/classfile.json'}, 'train_dataset': {'type': 'image_csv', 'csv_filename': 'results/TextLine/training_data/train.csv', 'base_dir': 'results/TextLine/training_data', 'repeat_dataset': 4, 'compose': {'transforms': [{'type': 'fixed_size_resize', 'output_size': 1000000.0}]}}, 'val_dataset': {'type': 'image_csv', 'csv_filename': 'results/TextLine/training_data/val.csv', 'base_dir': 'results/TextLine/training_data', 'compose': {'transforms': [{'type': 'fixed_size_resize', 'output_size': 1000000.0}]}}, 'model': {'encoder': 'resnet50', 'decoder': {'decoder_channels': [512, 256, 128, 64, 32], 'max_channels': 512}}, 'metrics': [['miou', 'iou'], ['iou', {'type': 'iou', 'average': None}], 'precision'], 'optimizer': {'lr': 0.0001}, 'lr_scheduler': {'type': 'exponential', 'gamma': 0.9995}, 'val_metric': '+miou', 'early_stopping': {'patience': 4}, 'model_out_dir': 'results/TextLine/model', 'num_epochs': 100, 'evaluate_every_epoch': 5,

epoch 0: loss=???:   0%|          | 0/100 [00:00<?, ?it/s]

iter=0: loss=???: 0it [00:00, ?it/s]

## Inference

In [ ]:
labels = ['TextLine']

In [ ]:
from deep_learning_lab import model
import os

predictor = model.Predictor(
    labels,
    input_dir= 'inference_data',
    output_dir= None,
    output_size= None,
    from_csv= os.path.join('training_data', 'test.csv'),
    reset_input= True
)

predictor.start(
    batch_size= 4,
    drawRegions= True,
    cutVignettes= True,
    bounding_box= False,
    verbose= True
)

## Tests

In [ ]:
# from matplotlib import pyplot as plt
from PIL import Image
import matplotlib.pyplot as plt

assert len(results)

In [ ]:
results[0].keys()

In [ ]:
image_nb = 0
predictions = results[image_nb]

In [ ]:
Image.fromarray(predictions['regions'])

In [ ]:
Image.fromarray(predictions['probasMaps'][1])

In [ ]:
for vignette in predictions['vignettes']:
    plt.imshow(Image.fromarray(vignette))
    plt.show()